## Задание 1
Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео.  
Нынешний таймспент на юзера в день в среднем равен 25 минут, а стандартная ошибка (SD) равна 156.   
Мы предполагаем, что в новой версии таймспент на юзера в день изменится на 10%.  
Средний трафик 20000 человек в день.   
Посчитайте сколько дней необходимо держать эксперимент при alpha = 5% и beta = 20% .

In [21]:
from typing import Union
from statsmodels.stats.power import tt_ind_solve_power

# Создаем функцию для расчета размера эффекта между двумя выборками
# с непрерывными признаками


def continuous_effect_size(mean1: Union[float, int],
                           std1: Union[float, int],
                           mean2: Union[float, int],
                           std2: Union[float, int],
                           nobs1: int = 10_000,
                           nobs2: int = 10_000) -> float:

    effect_size_formula = (mean1 - mean2) / ((std1**2 + std2**2) / 2) ** 0.5
    return effect_size_formula

# Создаем функцию для расчета необходимого размера выборок,
# требуемой статистической мощности и уровня значимости


def sample_size_continuous(effect_size: float,
                           alpha: float = .05,
                           beta: float = .2,
                           ratio: Union[float, int] = 1) -> int:

    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                           )
    return int(n * 2)


# Задаем параметры эксперимента (изменяем среднее и стандартное отклонение выборки
# теста на 10%)
mu_control, mu_test = 25, 25*1.10
std_control, std_test = 156, 156*1.10
traffic_per_day = 20000

# Вычисляем размер эффекта между выборками
effect_size = continuous_effect_size(
    mean1=mu_control, std1=std_control, mean2=mu_test, std2=std_test)

# Расчитываем размер выборки с требуемой статистической мощностью на заданном
# уровне значимости. Определяем, сколько дней нужно для получения достаточного
# трафика на сайте
traffic = sample_size_continuous(effect_size=effect_size, alpha=.05, beta=.2)
traffic_per_day = 20000
traffic_per_day_needed = traffic / traffic_per_day
print(traffic_per_day_needed)


6.7542


### Ответ: 
Из расчетов следует, что для того чтобы изменить продукт на главной странице сайта Mail.ru рекомендательной лентой коротких видео, необходимо проводить эксперимент не менее 7 дней. Такой срок позволит обеспечить надежные результаты статистического теста гипотезы о равенстве средних двух выборок (до и после изменения продукта). Использование в рекомендательной ленте коротких видео имеет потенциал повышения таймспента на сайте на 10%, что может положительно повлиять на активность и удержание пользователей. Однако, необходимо убедиться в значимости этого изменения, проведя эксперимент и проанализировав результаты.

## Задание 2
Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта.   
Одна из основных метрик bounce rate в GA = 40%. Мы предполагаем, что при оптимизации сайта она изменится минимум на 20%.   
Средний трафик 4000 человек в день. Посчитайте сколько нам нужно дней держать эксперимент при alpha = 5% и beta = 20%

In [22]:
from math import asin 
from statsmodels.stats.power import zt_ind_solve_power

# Создаем фуекцию для расчета эффекта между двумя долями конверсии

def calc_proportion_effect_size(conv_1: float, nobs_1: int, conv_2: float, nobs_2: int) -> float:
    prob_1, prob_2 = conv_1, conv_2
    es_formula = 2 * asin(prob_1**0.5) - 2 * asin(prob_2**0.5)
    return es_formula

# Создаем функцию для расчета необходимого размера выборки


def calc_sample_size_proportion(effect_size: float, alpha: float = .05, beta: float = .2, ratio: float = 1) -> int:
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha, power=(1 - beta), ratio=ratio)
    return int(n*2)


# Задаем начальные параметры
traffic_per_day = 4000  # общее количество посетителей в день
# доля конверсии на первой и второй странице соответственно
conv_1, conv_2 = 60/100, 80/100
# количество пользователей, покидающих первую страницу
one_page_users = 40 * traffic_per_day / 100
# количество наблюдений на первой и второй странице соответственно
nobs_1, nobs_2 = traffic_per_day, traffic_per_day
prob_1, prob_2 = conv_1, conv_2
# расчет эффекта между двумя долями конверсии
effect_size = calc_proportion_effect_size(conv_1, nobs_1, conv_2, nobs_2)
# расчет необходимого размера выборки
size = calc_sample_size_proportion(effect_size=effect_size, alpha=.05, beta=.2)

# Вывод результатов
print(
    f"Рекомендуемая выборка {size}, количество юзеров покидающих первую страницу в день {one_page_users:.0f}")


Рекомендуемая выборка 160, количество юзеров покидающих первую страницу в день 1600


Если оптимизация сайта изменится на 20% относительно 40%, то она будет составлять 32%. Предположим, что bounce rate после оптимизации составит 20%. Это означает, что нужно посчитать долю посетителей, которые не покинули сайт после первой страницы. Для этого нужно взять выборку из 160 пользователей, что составляет 10% от всех "отказников" за сутки, и проанализировать результаты эксперимента в течение одного дня. Если количество отказов сократится до 20%, то эксперимент можно считать успешным.
